## Importing libraries

In [1]:
import tensorflow_recommenders as tfrs
import tensorflow_recommenders_addons as tfra
import tensorflow_recommenders_addons.dynamic_embedding as de
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2023-03-16 19:03:44.646238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-16 19:03:44.646277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.8.3


## Checking how the table size changes with insert

# Notes:

* `de.embedding_lookup()` function does not insert keys into the CuckooHashTable. It simply returns a dynamically created set of defualts using a given initializer. 
  * If you want to see the table actually being updated, you'd need to trigger backpropagation through the returned variable of `de.embedding_lookup()`.

* `DynamicEmbeddingOptimizer()` calls `var.update_op()`, which will [insert the keys](https://github.com/tensorflow/recommenders-addons/blob/r0.5/tensorflow_recommenders_addons/dynamic_embedding/python/ops/dynamic_embedding_ops.py#L365)
  * This is called after applying gradients to variables as shown [here](https://github.com/tensorflow/recommenders-addons/blob/r0.5/tensorflow_recommenders_addons/dynamic_embedding/python/ops/dynamic_embedding_optimizer.py#L139)

In [5]:
tf.keras.backend.clear_session()
# source: https://github.com/tensorflow/recommenders-addons/blob/master/tensorflow_recommenders_addons/dynamic_embedding/python/ops/dynamic_embedding_variable.py

w = de.get_variable(
        name="dynamic_embeddings", initializer=tf.random_normal_initializer(), dim=128, init_size=1
)

@tf.function
def lookup(w, ids):
    lookup =  de.embedding_lookup(params=w, ids=ids)
    tf.print("w.tables[0].size():", w.tables[0].size())
    tf.print("lookup:", lookup.shape)
    return lookup

a = tf.constant(list(range(10)), dtype=tf.int64)
b = tf.constant(list(range(8, 13)), dtype=tf.int64)

a_lookup = lookup(w, a)
print(f"a_lookup.shape: {a_lookup.shape}")
b_lookup = lookup(w, b)
print(f"a_lookup.shape: {b_lookup.shape}")

2023-03-16 19:04:02.198324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 19:04:02.198373: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-16 19:04:02.485289: I ./tensorflow_recommenders_addons/dynamic_embedding/core/kernels/lookup_impl/lookup_table_op_cpu.h:279] HashTable on CPU is created on default mode: K=l, V=f, init_size=1


w.tables[0].size(): 0
lookup: TensorShape(None)
a_lookup.shape: (10, 128)
w.tables[0].size(): 0
lookup: TensorShape(None)
a_lookup.shape: (5, 128)


## Updating the underneath `CuckooHashTable`

Underneath this is doing something like,

```
keys_tensor = tf.constant(['a', 'b', 'c'])
vals_tensor = tf.constant([7, 8, 9], dtype=tf.int64)
input_tensor = tf.constant(['a', 'f'])

tablel = tfra.dynamic_embedding.CuckooHashTable(
    key_dtype=tf.string,
    value_dtype=tf.int64,
    default_value=[0]
)
table.insert(keys_tensor, vals_tensor)
table.lookup(input_tensor).numpy()
```

In [6]:
tf.keras.backend.clear_session()

w_dummy = de.get_variable(
        name="dynamic_embeddings", initializer=tf.random_normal_initializer(), dim=128, init_size=1
    )

a = tf.constant(list(range(10)), dtype=tf.int64)
b = tf.constant(list(range(8, 13)), dtype=tf.int64)

a_lookup, keys_exists = w_dummy.lookup(a, return_exists=True)
print(keys_exists)
print(w.size())
w_dummy.upsert(a, a_lookup)
print(w.size()) 
b_lookup, keys_exists = w_dummy.lookup(b, return_exists=True)
print(keys_exists)

tf.Tensor([False False False False False False False False False False], shape=(10,), dtype=bool)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor([ True  True False False False], shape=(5,), dtype=bool)


2023-03-16 19:04:07.087856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 19:04:07.087922: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-16 19:04:07.089398: I ./tensorflow_recommenders_addons/dynamic_embedding/core/kernels/lookup_impl/lookup_table_op_cpu.h:279] HashTable on CPU is created on default mode: K=l, V=f, init_size=1


## Using a RestrictPolicy to reduce the size of the table

In [10]:
tf.keras.backend.clear_session()

w_dummy = de.get_variable(
    name="dynamic_embeddings", 
    initializer=tf.random_normal_initializer(), 
    dim=128, 
    init_size=1, 
    restrict_policy=de.FrequencyRestrictPolicy
)


a = tf.constant(list(range(10)), dtype=tf.int64)
b = tf.constant(list(range(8, 13)), dtype=tf.int64)

a_lookup, keys_exists = w_dummy.lookup(a, return_exists=True)
w_dummy.upsert(a, a_lookup)
w_dummy.restrict_policy.apply_update(a)
print(w_dummy.size()) 

b_lookup, keys_exists = w_dummy.lookup(b, return_exists=True)
w_dummy.upsert(b, b_lookup)
w_dummy.restrict_policy.apply_update(b)
print(w_dummy.size()) 

#print(w_dummy.restrict_policy.status)

# Reduce the size of the table
w_dummy.restrict(5)
print(w_dummy.size())
#w_dummy.export()

tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(13, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)


2023-03-16 19:07:24.694006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-16 19:07:24.694061: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-03-16 19:07:24.695491: I ./tensorflow_recommenders_addons/dynamic_embedding/core/kernels/lookup_impl/lookup_table_op_cpu.h:157] HashTable on CPU is created on optimized mode: K=l, V=i, DIM=1, init_size=1
2023-03-16 19:07:24.696632: I ./tensorflow_recommenders_addons/dynamic_embedding/core/kernels/lookup_impl/lookup_table_op_cpu.h:279] HashTable on CPU is created on default 